In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
import seaborn as sns

# Soil Sensors

In [ ]:
soil = pd.read_csv('stuard_soil_data.csv') 

In [ ]:
# Data preparation
soil['humidity'] = pd.to_numeric(soil['humidity'], errors='coerce')  # Converts non-numeric values to NaN
soil = soil.dropna(subset=['humidity'])  # Drop rows with NaN in the 'humidity' column
soil['humidity'] = soil['humidity'].astype(float)  # Convert to float

print("Cleaned humidity column:")
print(soil['humidity'].head())

# Normalize the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(soil[['humidity']])

# Create sequences for Gradient Boosting
def create_sequences_gb(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length, :].flatten())  # Flatten sequence
        y.append(data[i + sequence_length, 0])  # Next time step
    return np.array(X), np.array(y)

sequence_length = 24  # Use past 24 hours to predict the next hour
X, y = create_sequences_gb(scaled_data, sequence_length)

# Reserve 2% as unused (not used in training or testing)
unused_percentage = 0.02
unused_data_size = int(len(X) * unused_percentage)
X_unused = X[-unused_data_size:]
y_unused = y[-unused_data_size:]

# Use the remaining data for training and testing
X = X[:-unused_data_size]
y = y[:-unused_data_size]

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
y_train = y_train.ravel()
y_test = y_test.ravel()
# Add Gaussian noise to the training data
noise = np.random.normal(0, 0.01, X_train.shape)
X_train_noisy = X_train + noise

# Define the Gradient Boosting model and parameter grid
gb_model = GradientBoostingRegressor(random_state=42)
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
}

# Perform grid search cross-validation
grid_search = GridSearchCV(estimator=gb_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train_noisy, y_train)

# Best parameters and model
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")
best_modelH = grid_search.best_estimator_

# Evaluate on the test set
y_test_pred = best_modelH.predict(X_test)
y_test_pred_rescaled = scaler.inverse_transform(y_test_pred.reshape(-1, 1))
y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))

mse_test = mean_squared_error(y_test_rescaled, y_test_pred_rescaled)
mae_test = mean_absolute_error(y_test_rescaled, y_test_pred_rescaled)
r2_test = r2_score(y_test_rescaled, y_test_pred_rescaled)

print(f"Test Set Evaluation:\nMSE: {mse_test:.4f}\nMAE: {mae_test:.4f}\nR²: {r2_test:.4f}")

# Evaluate on unused data
y_unused_pred = best_modelH.predict(X_unused)
y_unused_pred_rescaled = scaler.inverse_transform(y_unused_pred.reshape(-1, 1))
y_unused_rescaled = scaler.inverse_transform(y_unused.reshape(-1, 1))

mse_unused = mean_squared_error(y_unused_rescaled, y_unused_pred_rescaled)
mae_unused = mean_absolute_error(y_unused_rescaled, y_unused_pred_rescaled)
r2_unused = r2_score(y_unused_rescaled, y_unused_pred_rescaled)

print(f"Unused Data Evaluation:\nMSE: {mse_unused:.4f}\nMAE: {mae_unused:.4f}\nR²: {r2_unused:.4f}")

# Visualize predictions vs actual values
plt.figure(figsize=(10, 5))
plt.plot(y_test_rescaled[:100], label='Actual Humidity')
plt.plot(y_test_pred_rescaled[:100], label='Predicted Humidity')
plt.title('Predicted vs Actual Humidity')
plt.xlabel('Time Steps')
plt.ylabel('Humidity')
plt.legend()
plt.show()


In [ ]:
# Step 1: Make predictions using the unused data
y_unused_pred =best_modelH.predict(X_unused)

# Step 2: Rescale predictions and actual values
y_unused_pred_rescaled = scaler.inverse_transform(y_unused_pred.reshape(-1, 1))
y_unused_rescaled = scaler.inverse_transform(y_unused.reshape(-1, 1))


mse = mean_squared_error(y_unused_rescaled, y_unused_pred_rescaled)
mae = mean_absolute_error(y_unused_rescaled, y_unused_pred_rescaled)
r2 = r2_score(y_unused_rescaled, y_unused_pred_rescaled)

# Step 4: Print error metrics
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"R-squared (R²): {r2:.4f}")

# Step 3: Plot the predicted vs. actual values
plt.figure(figsize=(10, 5))
plt.plot(y_unused_rescaled[:100], label='Actual Humidity')
plt.plot(y_unused_pred_rescaled[:100], label='Predicted Humidity')
plt.title('Predicted vs Actual Humidity for Unused Data')
plt.xlabel('Time Steps')
plt.ylabel('Humidity')
plt.legend()
plt.show()

# Optional: Print out some of the predicted values along with the actual values
print("Predicted vs Actual (First 50 samples):")
for i in range(50):
    print(f"Actual: {y_unused_rescaled[i][0]:.3f}, Predicted: {y_unused_pred_rescaled[i][0]:.3f}")


In [ ]:
joblib.dump(best_modelH, 'Humidity_model.joblib')

joblib.dump(X_test[0],'initial_sequenceH.joblib')

In [ ]:
# Step 1: Clean and preprocess the 'temperature' column
soil['temperature'] = pd.to_numeric(soil['temperature'], errors='coerce')  # Convert non-numeric to NaN
soil = soil.dropna(subset=['temperature'])  # Drop rows with NaN in 'temperature'
soil['temperature'] = soil['temperature'].astype(float)  # Ensure 'temperature' is a float

print("Cleaned temperature column:")
print(soil['temperature'].head())

# Step 2: Normalize the temperature data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(soil[['temperature']])

# Step 3: Create sequences for Gradient Boosting Regressor
def create_sequences_gbr(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length, :].flatten())  # Flatten the sequence
        y.append(data[i + sequence_length, :])  # Target is the next time step
    return np.array(X), np.array(y)

sequence_length = 24  # Use past 24 hours to predict the next hour
X, y = create_sequences_gbr(scaled_data, sequence_length)

# Step 4: Split into 98% usable data and 2% unused
unused_percentage = 0.02
unused_data_size = int(len(X) * unused_percentage)

X_unused = X[-unused_data_size:]
y_unused = y[-unused_data_size:]

X = X[:-unused_data_size]
y = y[:-unused_data_size]

# Split 98% data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
y_train = y_train.ravel()
y_test = y_test.ravel()

# Step 5: Perform Hyperparameter Tuning using GridSearchCV
gbr = GradientBoostingRegressor(random_state=42)

param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0]
}

grid_search = GridSearchCV(estimator=gbr, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(X_train, y_train)

# Get the best model and its parameters
best_gbrT = grid_search.best_estimator_
print(f"Best parameters: {grid_search.best_params_}")

# Step 6: Train the optimized Gradient Boosting Regressor
best_gbrT.fit(X_train, y_train)

# Step 7: Predict on the test set and unused data
y_test_pred = best_gbrT.predict(X_test)
y_test_pred_rescaled = scaler.inverse_transform(y_test_pred.reshape(-1, 1))
y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))

# Predict for unused data
y_unused_pred = best_gbrT.predict(X_unused)
y_unused_pred_rescaled = scaler.inverse_transform(y_unused_pred.reshape(-1, 1))
y_unused_rescaled = scaler.inverse_transform(y_unused.reshape(-1, 1))

# Step 8: Evaluate model performance on the test set
mse_test = mean_squared_error(y_test_rescaled, y_test_pred_rescaled)
mae_test = mean_absolute_error(y_test_rescaled, y_test_pred_rescaled)
r2_test = r2_score(y_test_rescaled, y_test_pred_rescaled)

print(f"Test Set Evaluation:\nMSE: {mse_test:.4f}\nMAE: {mae_test:.4f}\nR²: {r2_test:.4f}")

# Step 9: Evaluate model performance on the unused data
mse_unused = mean_squared_error(y_unused_rescaled, y_unused_pred_rescaled)
mae_unused = mean_absolute_error(y_unused_rescaled, y_unused_pred_rescaled)
r2_unused = r2_score(y_unused_rescaled, y_unused_pred_rescaled)

print(f"Unused Data Evaluation:\nMSE: {mse_unused:.4f}\nMAE: {mae_unused:.4f}\nR²: {r2_unused:.4f}")

# Step 10: Visualize predicted vs. actual temperature for test set
plt.figure(figsize=(10, 5))
plt.plot(y_test_rescaled[:100], label='Actual Temperature')
plt.plot(y_test_pred_rescaled[:100], label='Predicted Temperature')
plt.title('Predicted vs Actual Temperature for Test Set')
plt.xlabel('Time Steps')
plt.ylabel('Temperature')
plt.legend()
plt.show()

# Visualize predicted vs. actual temperature for unused data
plt.figure(figsize=(10, 5))
plt.plot(y_unused_rescaled[:100], label='Actual Temperature (Unused)')
plt.plot(y_unused_pred_rescaled[:100], label='Predicted Temperature (Unused)')
plt.title('Predicted vs Actual Temperature for Unused Data')
plt.xlabel('Time Steps')
plt.ylabel('Temperature')
plt.legend()
plt.show()

In [ ]:
best_gbrT=joblib.dump(best_modelH, 'Temperature_model.joblib')

joblib.dump(X_test[0],'initial_sequenceT.joblib')

condectivety


In [ ]:
soil = pd.read_csv('stuard_soil_data.csv') 
# Data preparation
soil['electrical_conductivity'] = pd.to_numeric(soil['electrical_conductivity'], errors='coerce')  # Converts non-numeric values to NaN
soil = soil.dropna(subset=['electrical_conductivity'])  # Drop rows with NaN in the 'electrical_conductivity' column
soil['electrical_conductivity'] = soil['electrical_conductivity'].astype(float)  # Convert to float

print("Cleaned electrical_conductivity column:")
print(soil['electrical_conductivity'].head())

# Normalize the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(soil[['electrical_conductivity']])

# Create sequences for Gradient Boosting
def create_sequences_gb(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length, :].flatten())  # Flatten sequence
        y.append(data[i + sequence_length, 0])  # Next time step
    return np.array(X), np.array(y)

sequence_length = 24  # Use past 24 hours to predict the next hour
X, y = create_sequences_gb(scaled_data, sequence_length)

# Reserve 2% as unused (not used in training or testing)
unused_percentage = 0.02
unused_data_size = int(len(X) * unused_percentage)
X_unused = X[-unused_data_size:]
y_unused = y[-unused_data_size:]

# Use the remaining data for training and testing
X = X[:-unused_data_size]
y = y[:-unused_data_size]

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
y_train = y_train.ravel()
y_test = y_test.ravel()
# Add Gaussian noise to the training data
noise = np.random.normal(0, 0.01, X_train.shape)
X_train_noisy = X_train + noise

# Define the Gradient Boosting model and parameter grid
gb_model = GradientBoostingRegressor(random_state=42)
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0]
}

# Perform grid search cross-validation
grid_search = GridSearchCV(estimator=gb_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train_noisy, y_train)

# Best parameters and model
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")
best_model = grid_search.best_estimator_

# Evaluate on the test set
y_test_pred = best_model.predict(X_test)
y_test_pred_rescaled = scaler.inverse_transform(y_test_pred.reshape(-1, 1))
y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))

mse_test = mean_squared_error(y_test_rescaled, y_test_pred_rescaled)
mae_test = mean_absolute_error(y_test_rescaled, y_test_pred_rescaled)
r2_test = r2_score(y_test_rescaled, y_test_pred_rescaled)

print(f"Test Set Evaluation:\nMSE: {mse_test:.4f}\nMAE: {mae_test:.4f}\nR²: {r2_test:.4f}")

# Evaluate on unused data
y_unused_pred = best_model.predict(X_unused)
y_unused_pred_rescaled = scaler.inverse_transform(y_unused_pred.reshape(-1, 1))
y_unused_rescaled = scaler.inverse_transform(y_unused.reshape(-1, 1))

mse_unused = mean_squared_error(y_unused_rescaled, y_unused_pred_rescaled)
mae_unused = mean_absolute_error(y_unused_rescaled, y_unused_pred_rescaled)
r2_unused = r2_score(y_unused_rescaled, y_unused_pred_rescaled)

print(f"Unused Data Evaluation:\nMSE: {mse_unused:.4f}\nMAE: {mae_unused:.4f}\nR²: {r2_unused:.4f}")

# Visualize predictions vs actual values
plt.figure(figsize=(10, 5))
plt.plot(y_test_rescaled[:100], label='Actual electrical_conductivity')
plt.plot(y_test_pred_rescaled[:100], label='Predicted electrical_conductivity')
plt.title('Predicted vs Actual electrical_conductivity')
plt.xlabel('Time Steps')
plt.ylabel('electrical_conductivity')
plt.legend()
plt.show()


In [ ]:
y_unused_pred = best_model.predict(X_unused)
y_unused_pred_rescaled = scaler.inverse_transform(y_unused_pred.reshape(-1, 1))
y_unused_rescaled = scaler.inverse_transform(y_unused.reshape(-1, 1))

mse_unused = mean_squared_error(y_unused_rescaled, y_unused_pred_rescaled)
mae_unused = mean_absolute_error(y_unused_rescaled, y_unused_pred_rescaled)
r2_unused = r2_score(y_unused_rescaled, y_unused_pred_rescaled)

print(f"Unused Data Evaluation:\nMSE: {mse_unused:.4f}\nMAE: {mae_unused:.4f}\nR²: {r2_unused:.4f}")

# Visualize predictions vs actual values
plt.figure(figsize=(10, 5))
plt.plot(y_unused_rescaled[:100], label='Actual electrical_conductivity')
plt.plot(y_unused_pred_rescaled[:100], label='Predicted electrical_conductivity')
plt.title('Predicted vs Actual electrical_conductivity')
plt.xlabel('Time Steps')
plt.ylabel('electrical_conductivity')
plt.legend()
plt.show()

In [ ]:

joblib.dump(best_model, 'condic_model.joblib')

joblib.dump(X_test[0],'initial_sequenceD.joblib')

# Environmental Sonsores

In [ ]:
env=pd.read_csv('stuard_environmental_data.csv')

In [ ]:
# Data preparation
env['env_humidity'] = pd.to_numeric(env['env_humidity'], errors='coerce')  # Converts non-numeric values to NaN
env = env.dropna(subset=['env_humidity'])  # Drop rows with NaN in the 'env_humidity' column
env['env_humidity'] = env['env_humidity'].astype(float)  # Convert to float

print("Cleaned env_humidity column:")
print(env['env_humidity'].head())

# Normalize the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(env[['env_humidity']])

# Create sequences for Gradient Boosting
def create_sequences_gb(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length, :].flatten())  # Flatten sequence
        y.append(data[i + sequence_length, 0])  # Next time step
    return np.array(X), np.array(y)

sequence_length = 24  # Use past 24 hours to predict the next hour
X, y = create_sequences_gb(scaled_data, sequence_length)

# Reserve 2% as unused (not used in training or testing)
unused_percentage = 0.02
unused_data_size = int(len(X) * unused_percentage)
X_unused = X[-unused_data_size:]
y_unused = y[-unused_data_size:]

# Use the remaining data for training and testing
X = X[:-unused_data_size]
y = y[:-unused_data_size]

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
y_train = y_train.ravel()
y_test = y_test.ravel()
# Add Gaussian noise to the training data
noise = np.random.normal(0, 0.01, X_train.shape)
X_train_noisy = X_train + noise

# Define the Gradient Boosting model and parameter grid
gb_model = GradientBoostingRegressor(random_state=42)
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
}

# Perform grid search cross-validation
grid_search = GridSearchCV(estimator=gb_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train_noisy, y_train)

# Best parameters and model
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")
best_modelEH = grid_search.best_estimator_

# Evaluate on the test set
y_test_pred = best_modelH.predict(X_test)
y_test_pred_rescaled = scaler.inverse_transform(y_test_pred.reshape(-1, 1))
y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))

mse_test = mean_squared_error(y_test_rescaled, y_test_pred_rescaled)
mae_test = mean_absolute_error(y_test_rescaled, y_test_pred_rescaled)
r2_test = r2_score(y_test_rescaled, y_test_pred_rescaled)

print(f"Test Set Evaluation:\nMSE: {mse_test:.4f}\nMAE: {mae_test:.4f}\nR²: {r2_test:.4f}")

# Evaluate on unused data
y_unused_pred = best_modelH.predict(X_unused)
y_unused_pred_rescaled = scaler.inverse_transform(y_unused_pred.reshape(-1, 1))
y_unused_rescaled = scaler.inverse_transform(y_unused.reshape(-1, 1))

mse_unused = mean_squared_error(y_unused_rescaled, y_unused_pred_rescaled)
mae_unused = mean_absolute_error(y_unused_rescaled, y_unused_pred_rescaled)
r2_unused = r2_score(y_unused_rescaled, y_unused_pred_rescaled)

print(f"Unused Data Evaluation:\nMSE: {mse_unused:.4f}\nMAE: {mae_unused:.4f}\nR²: {r2_unused:.4f}")

# Visualize predictions vs actual values
plt.figure(figsize=(10, 5))
plt.plot(y_test_rescaled[:100], label='Actual env_humidity')
plt.plot(y_test_pred_rescaled[:100], label='Predicted env_humidity')
plt.title('Predicted vs Actual env_humidity')
plt.xlabel('Time Steps')
plt.ylabel('env_humidity')
plt.legend()
plt.show()


In [ ]:
joblib.dump(best_modelEH, 'condic_model.joblib')

joblib.dump(X_test[0],'initial_sequenceD.joblib')

In [ ]:
# Data preparation
env['env_Temperature'] = pd.to_numeric(env['env_Temperature'], errors='coerce')  # Converts non-numeric values to NaN
env = env.dropna(subset=['env_Temperature'])  # Drop rows with NaN in the 'env_Temperature' column
env['env_Temperature'] = env['env_Temperature'].astype(float)  # Convert to float

print("Cleaned env_Temperature column:")
print(env['env_Temperature'].head())

# Normalize the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(env[['env_Temperature']])

# Create sequences for Gradient Boosting
def create_sequences_gb(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length, :].flatten())  # Flatten sequence
        y.append(data[i + sequence_length, 0])  # Next time step
    return np.array(X), np.array(y)

sequence_length = 24  # Use past 24 hours to predict the next hour
X, y = create_sequences_gb(scaled_data, sequence_length)

# Reserve 2% as unused (not used in training or testing)
unused_percentage = 0.02
unused_data_size = int(len(X) * unused_percentage)
X_unused = X[-unused_data_size:]
y_unused = y[-unused_data_size:]

# Use the remaining data for training and testing
X = X[:-unused_data_size]
y = y[:-unused_data_size]

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
y_train = y_train.ravel()
y_test = y_test.ravel()
# Add Gaussian noise to the training data
noise = np.random.normal(0, 0.01, X_train.shape)
X_train_noisy = X_train + noise

# Define the Gradient Boosting model and parameter grid
gb_model = GradientBoostingRegressor(random_state=42)
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
}

# Perform grid search cross-validation
grid_search = GridSearchCV(estimator=gb_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train_noisy, y_train)

# Best parameters and model
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")
best_modelET = grid_search.best_estimator_

# Evaluate on the test set
y_test_pred = best_modelH.predict(X_test)
y_test_pred_rescaled = scaler.inverse_transform(y_test_pred.reshape(-1, 1))
y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))

mse_test = mean_squared_error(y_test_rescaled, y_test_pred_rescaled)
mae_test = mean_absolute_error(y_test_rescaled, y_test_pred_rescaled)
r2_test = r2_score(y_test_rescaled, y_test_pred_rescaled)

print(f"Test Set Evaluation:\nMSE: {mse_test:.4f}\nMAE: {mae_test:.4f}\nR²: {r2_test:.4f}")

# Evaluate on unused data
y_unused_pred = best_modelH.predict(X_unused)
y_unused_pred_rescaled = scaler.inverse_transform(y_unused_pred.reshape(-1, 1))
y_unused_rescaled = scaler.inverse_transform(y_unused.reshape(-1, 1))

mse_unused = mean_squared_error(y_unused_rescaled, y_unused_pred_rescaled)
mae_unused = mean_absolute_error(y_unused_rescaled, y_unused_pred_rescaled)
r2_unused = r2_score(y_unused_rescaled, y_unused_pred_rescaled)

print(f"Unused Data Evaluation:\nMSE: {mse_unused:.4f}\nMAE: {mae_unused:.4f}\nR²: {r2_unused:.4f}")

# Visualize predictions vs actual values
plt.figure(figsize=(10, 5))
plt.plot(y_test_rescaled[:100], label='Actual env_Temperature')
plt.plot(y_test_pred_rescaled[:100], label='Predicted env_Temperature')
plt.title('Predicted vs Actual env_Temperature')
plt.xlabel('Time Steps')
plt.ylabel('env_Temperature')
plt.legend()
plt.show()


In [ ]:
joblib.dump(best_modelET, 'condic_model.joblib')

joblib.dump(X_test[0],'initial_sequenceD.joblib')